In [1]:
import os

In [2]:
pwd

'c:\\Users\\arkab\\Documents\\Chest-Cancer-Classification\\research'

In [3]:
#so wee need to go back one folder  

In [4]:
os.chdir("../")

In [5]:
pwd

'c:\\Users\\arkab\\Documents\\Chest-Cancer-Classification'

In [6]:
#creating entity 

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
#Update the configuration manager in src config

In [9]:
#importing constants in file
from classifier.constants import *
from classifier.utils.common import read_yaml,create_directories

In [10]:
#trying to read contentns of config.yaml file and store it in a DataIngestionConfig which can be later used

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

   #Method will create directory inside artifact folder and storing all data ingestion folder details in a object 
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config


In [12]:
import os
import zipfile
import gdown
from classifier import logger
from classifier.utils.common import get_size

In [13]:
#creating data ingestion component

In [17]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config


    def download(self)->str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix='https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir} ")

        except Exception as e:
            raise e;

    def extract_zip_file(self):
        """
        Extracts zip file into the data directory
        Method returns none
        """

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)    

In [15]:
#update pipeline to execute 

In [19]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    


[2024-01-12 00:22:23,774:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-12 00:22:23,778:INFO:common:yaml file: params.yaml loaded successfully]
created directory at : artifacts
created directory at : artifacts/data_ingestion
[2024-01-12 00:22:23,781:INFO:4156986932:Downloading data from https://drive.google.com/drive/u/0/folders/1_xtcrc9_N6CopyesHqN5YJ18KUC0EIpJ into file artifacts/data_ingestion/data.zip]


MissingSchema: Invalid URL '1_xtcrc9_N6CopyesHqN5YJ18KUC0EIp': No scheme supplied. Perhaps you meant https://1_xtcrc9_N6CopyesHqN5YJ18KUC0EIp?